# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: I used **LoRA** as my PEFT technique. LoRA is the only PEFT technique that is compatible with all models at this time.
* Model: I used **distilbert-base-uncased** as my model. This is a relatively small model that is compatible with sequence classification and **LoRA**.
* Evaluation approach: The evaluation approach covered in this project was the `evaluate` method with a **Hugging Face** `Trainer`.
* Fine-tuning dataset: I use a dataset from Hugging Face's datasets library [stanfordnlp/imdb](https://huggingface.co/datasets/stanfordnlp/imdb).

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

### Loading the model

In [2]:
# See: https://huggingface.co/datasets/stanfordnlp/imdb
dataset = load_dataset(
    "stanfordnlp/imdb", split="train"
).train_test_split(test_size=0.2, shuffle=True, seed=23)

splits = ["train", "test"]
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})

In [3]:
# Inspect the first example
dataset["train"][0]

{'text': 'The stories in this video are very entertaining, and it definately is worth a look! The first one concerns a young couple harrassed in the woods by two rednecks, with a great, but unexplained twist at the end.<br /><br />The seond is the best of the lot, and it alone, makes this worth watching - A man is attacked by a dog, which he fears to be rabid - He finds shelter in what appears to be a hospital, but he finds out the employees there are not exactly what they appear to be...... Great twist at the end, and this episode alone scores 10/10! If the others were up to par with this one, this would get 10/10!<br /><br />The third is the weakest of the bunch - A girl meets with some guys and has wild sex! There appears to be no point to the story until the end, with a good little twist, but it is spoiled by the awful first part!<br /><br />Never the less, this is a great movie that will not do you wrong at all! Well worth a rental!',
 'label': 1}

### Pre-process datasets

Now we are going to process our datasets by converting all the text into tokens for our models.

In [ ]:
# Loading the Tokenizer and tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

tokenized_dataset["train"]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 20000
})

### Load and set up the model

In this case we are doing a full fine tuning, so we will want to unfreeze all parameters.

In [5]:
# Loading the pretrained HuggingFace foundation model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Unfreeze all the model parameters
for param in model.parameters():
    param.requires_grad = True

In [7]:
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Let's train it!

Now it's time to train our model. We'll use the `Trainer` class.

First we'll define a function to compute our accuracy metreic then we make the `Trainer`.

In this instance, we will fill in some of the training arguments

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./model",
        # Set the learning rate
        learning_rate=2e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        # Set the learning rate
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.693300,0.693129,0.503800
2,0.693200,0.693134,0.503800


TrainOutput(global_step=10000, training_loss=0.6984223693847657, metrics={'train_runtime': 1937.87, 'train_samples_per_second': 20.641, 'train_steps_per_second': 5.16, 'total_flos': 4437059681878944.0, 'train_loss': 0.6984223693847657, 'epoch': 2.0})

### Evaluate the model

Evaluating the model is as simple as calling the evaluate method on the trainer object. This will run the model on the test set and compute the metrics we specified in the compute_metrics function.

In [9]:
# Show the performance of the model on the test set
# What do you think the evaluation accuracy will be?
trainer.evaluate()

{'eval_loss': 0.6931286454200745,
 'eval_accuracy': 0.5038,
 'eval_runtime': 69.2983,
 'eval_samples_per_second': 72.152,
 'eval_steps_per_second': 18.038,
 'epoch': 2.0}

### View the results

Let's look at a few examples

In [10]:
# Make a dataframe with the predictions and the text and the labels
items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "text": [item["text"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,text,predictions,labels
0,"As soon as I heard about this film I knew I had to check it out. Well, I heard about it, then I found the trailer. After that, that's when I knew I had to see it. And I am so glad I did. You want to see classic television mixed with zombies? No? Then get lost.<br /><br />FIDO is a movie unlike anything I've ever seen. Well, actually, it kind of is. It's kind of like a Lassie episode and a Zombie film. Though when combined, it feels completely new and original. FIDO is about a little boy named Timmy and his new pet Fido. Well this new pet ain't no squawking parakeet or some potty-trained puppy. It's a re-animated dead guy...a zombie. A large radiation cloud engulfed Earth which led to all of the dead rising, which ensued the Zombie Wars. Though through the genius of Reinhold Giger, lead scientist of ZomCon, he discovered that if you destroy the brain, the zombie will perish, thus giving us the edge and the win in the Zombie War. Though due to lingering radiation, whoever dies becomes a zombie. Which can be a problem especially with the elderly. Though Zomcom steps up again with more breakthroughs, especially with the Domestication Collar. The collar stops the zombie's need for human flesh and thus making it harmless as a household pet. But not all is perfect in this Zombie Utopia, collars break, old people die and....well I'll just let you watch this incredibly unique flick.<br /><br />FIDO is a fantastic idea brought to fruition. With an all-star cast, and great writing FIDO rises above most in the comedy/horror genre. There are plenty of funny and original situations that really had me entertained. Though after seeing the film, I personally think the movie would have been better in black and white. At less than 90 minutes, the movie doesn't go on for too long and moves from scene to scene at a good rate. It'll probably end up being a cult-classic of sorts, since it's not really a laugh out loud comedy or even a horror movie. It's a comedy/family/zombie film immersed in the 1950 vibe. If you thought anything I said here was interesting by all means check this film out. But if you're still on the fence, swing your leg back over and stay there. 8.5 outta 10",1,1
1,"When his in-laws are viciously murdered by a gang of thugs, a young deputy is ordered to escort his mute friend, forced to take the rap by the gang, to Tucson for trial and ending up having to face the real killers along the way.<br /><br />The Decoy is a real-life decoy sent to video stores to lure you away from better films! It's talky, illogical, slow, and ultimately very boring.<br /><br />There's some good costumes, sets, and photography but nothing else is good about this vanity project from writer/director/producer/star Justin Kreinbrink, who apparently had too much money on his hands.<br /><br />They used to make westerns like this, that were under an hour long. Trim this of about half it's length and you might have something watchable.",1,0
2,"This is not a good movie. It's disjointed, all the acting is bad, and has a lame story you've seen a thousand times done much better else where. Not to mention you can see every plot point coming from a mile away. Worst of all, no one bothered to tell Lonette Mckee she can't sing. But who cares, she's sooooo damn good looking. But I digress, nothing new here. Bottom-line, hot girl group gets taken advantage of, some one gets hooked on drugs, someone gets hooked on a guy, some one gets the hell out, and then the horrible stuff happens. Surprise, surprise. Welcome to the music business. I can't believe so many people out there think this is a good movie. So many of you seem to want to use a sliding scale when it comes to grading Black Movies. I don't play that! If you want to support these films by going to see them - great! If you enjoyed it - super! To each his own. But don't try to tell me it was good. Pleeeease! I wish colored folks would not fawn over these kind of movies just because they

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.